### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42330975,79085,51399,51530,0,0,1128,5,2025-02-20 22:00,2025-02-20 22:00,6,0,1699,5,1
34,42330278,28958,79085,3,0,0,1128,6,2025-02-22 00:29,2025-02-22 00:30,0,0,6438,3,1
35,42334375,77240,79085,51530,0,0,1129,0,2025-02-22 19:30,2025-02-22 19:30,1,1,2928,4,1
36,42084889,79085,89569,1349,19,69,1129,1,2025-02-23 21:59,2025-02-23 21:59,1,4,12854,6,1
37,42334364,79085,36137,51530,0,0,1129,2,2025-02-24 22:00,2025-02-24 22:00,0,3,4268,4,1
38,42334637,79085,86975,51530,0,0,1129,3,2025-02-25 22:00,2025-02-25 22:00,1,0,2341,2,1
39,42084873,79085,78671,1349,20,69,1129,4,2025-02-26 21:59,2025-02-26 21:59,0,1,12501,2,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,31278,35288,4,28,2,0,0,4-2-4a,11,9,10,42330975
1,51399,38792,34782,2,3,3,1,0,HUSARIA,6,8,7,42330975
0,28958,28096,36716,1,8,2,1,0,NARANJA 23,8,6,7,42330278
1,79085,42320,33700,1,1,1,0,0,5-4-1a,5,7,6,42330278
0,77240,38703,37720,0,4,3,1,0,check,8,9,11,42334375
1,79085,31275,32258,0,9,1,0,0,3-4-3BC,10,9,10,42334375
0,79085,32691,34363,1,9,0,0,0,4-3-3a,10,9,10,42084889
1,89569,37260,35588,1,8,3,1,0,442 new,10,6,12,42084889
0,79085,41140,35424,2,4,1,1,0,5-4-1a,9,9,10,42334364
1,36137,28814,34530,0,8,1,0,0,Miðja,11,8,9,42334364


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1588

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,20,55,18,1,1,98,16,55582098018010,1349
1,6295,20,47,15,2,3,103,33,47570103015009,1349
2,23755,20,43,13,4,3,92,29,43563092013011,1349
3,79866,20,41,13,2,5,56,22,41534056013005,1349
4,78671,20,38,12,2,6,67,25,38542067012007,1349
5,89569,20,31,10,1,9,47,47,31500047010002,1349
6,124139,20,28,9,1,10,56,55,28501056009008,1349
7,122169,20,26,8,2,10,87,73,26514087008004,1349
8,79085,20,26,7,5,8,43,30,26513043007006,1349
9,48541,20,9,3,0,17,21,125,9396021003001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()